In [6]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()


 ········


In [47]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from IPython.display import display, Latex
model = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")

東京大学2023年入試問題\
黒玉3個、赤玉4個、白玉5個が入っている袋から玉を1個ずつ取り出し、取り出した玉を順に横一列に12個すべて並べる。ただし、袋から個々の玉が取り出される確率は等しいものとする。\
どの赤玉も隣り合わない確率pを求めよ\

解答:全ての球を区別すると並べ方は１２！通り。赤玉が隣りあわない並べ方は、まず赤玉以外の並べ方が８！通り、これらの球の間もしくは両端の９ヶ所のどこにどの赤玉を入れるかを考えると9・8・7・6・5通り。\
よって、p=(8!9・8・7・6・5)/12!=14/55

まずはgpt-3.5-turboに普通に解かせてみる。

In [48]:
def solver_func(system_message,question):  
    message = [SystemMessage(system_message),
              HumanMessage(question)]
    return display(Latex(model.invoke(message).content))

In [49]:
system_message = "与えられた問題を解け。導出過程も説明すること。"
question = "　黒玉3個、赤玉4個、白玉5個が入っている袋から玉を1個ずつ取り出し、取り出した玉を順に横一列に12個すべて並べる。ただし、袋から個々の玉が取り出される確率は等しいものとする。\
どの赤玉も隣り合わない確率pを求めよ。"
solver_func(system_message, question)

<IPython.core.display.Latex object>

誤り。一文一文よく考えさせてみる。

In [50]:
system_message = "与えられた問題を解け。一文一文よく考えて解くこと。導出過程も説明すること。"
solver_func(system_message, question)

<IPython.core.display.Latex object>

誤り。誤りである原因は、前の文で自分が述べた内容を忘れること(初めは球の順番を考慮しているのに、後の方になると忘れている)。\
球同士を区別するしないの一貫性もない。そこでプロンプトを以下のように変更。



In [53]:
system_message = "与えられた問題を解け。一文書き終えて新しい文を書き始めるたびに、それまで自分が書いた文と問題文を思い出し、\
論理的一貫性を失わぬようにすること。導出過程も説明すること。"
solver_func(system_message, question)

<IPython.core.display.Latex object>

誤り。計算も間違えている。一回しか解かないでそれを結論とするのが問題なのかもしれない。違う方法でとかせ見直しさせてみる。

In [54]:
system_message = "次の順番で与えられた問題を解け。\
手順１：与えられた問題を解け。一文書き終えて新しい文を書き始めるたびに、それまで自分が書いた文と問題文を思い出し、\
論理的一貫性を失わぬようにすること。導出過程も説明すること。\
手順２：手順１とは異なる方法で与えられた問題を解け。一文書き終えて新しい文を書き始めるたびに、それまで自分が書いた文と問題文を思い出し、\
論理的一貫性を失わぬようにすること。導出過程も説明すること。\
手順３：手順１と手順２の解答を比較する。手順１と手順２の解答が同じならそれを解答として良い。手順１と手順２の解答が違うならば、いずれの解答がより論理的一貫性があるかを判定し、判定理由も与えよ。\
より論理的一貫性のある方を解答とせよ。\
以下のフォーマットに従って解答すること：\
手順１:{ここに手順１}\
手順２:{ここに手順２}\
手順３:{ここに手順３}"
solver_func(system_message, question)

<IPython.core.display.Latex object>

誤り。見たところ、各文で述べている内容は正しいのだが、それを最後の分数式に落とし込む段階で論理的でない判断をしているようだ。英語の方がLLMの精度が高いはずだから、英語でプロンプトを入れてみる。

In [58]:
system_message = "You are a renowned mathematician. Solve the given question step by step. Make sure you think through every sentence you write,\
and that every sentence you write is coherent with all the previous sentences and the setting of the problem."
question = "Remove one ball at a time from a bag containing three black balls, four red balls and five white balls, and arrange all 12 balls in a horizontal line in the order in which they were removed. \
Note that the probability of an individual ball being taken out of the bag shall be equal. Find the probability p that none of the red balls are adjacent to each other."
solver_func(system_message, question)

<IPython.core.display.Latex object>

惜しい。球同士の区別は保っている。四つの赤玉のうち二つだけ隣り合わせになる可能性を考慮していないため間違えたもの。

In [60]:
system_message = "Solve the given problems in the following order. \
Step 1: Solve the given problems. Each time you finish a sentence and start a new sentence, recall the sentences you have written up to that point \
and the problem sentence, and Ensure that you do not lose logical coherence. The derivation process should also be explained. \
Step 2: Solve the given problem in a different way from step 1. Each time you finish writing a sentence and start writing a new sentence, \
recall the sentence you have written and the problem sentence up to that point and Ensure that logical coherence is not lost.\
The derivation process should also be explained. \
Step 3: Compare the answers to steps 1 and 2. If the answers to steps 1 and 2 are the same, then use that as your answer.\
If the answers to steps 1 and 2 are different, determine which answer is more logically consistent and take the more logically consistent one as the answer, \
and give the reason for thinking so.\
Answer according to the following format:\
Step 1:{Step 1 here}\
Step 2:{Step 2 here}\
Step 3:{Step 3 here}\
final answer:{Answer in one number}"
solver_func(system_message, question)

<IPython.core.display.Latex object>

間違い。そもそもstep3で明らかに違う回答を同じと言っているから、step3は本当に機能していないし、分数計算もできていないようだ。

考察:\
gpt３.5turboは英語にしようが日本語にしようがこの手の数学の問題は苦手なようだ。複数の方法を解いて見直させようにも、どのようにしてどちらの解が優れているのか判定すれば\
いいかわからず、とりあえず今回はgptに自分で判定するよう丸投げしたが、それもこのモデルではできないようだ。そして、計算の精度も低い。

追加:gpt-4o-mini-2024-07-18でやってみる。

In [62]:
model = ChatOpenAI(temperature=0, model="gpt-4o-mini-2024-07-18")
system_message = "You are a renowned mathematician. Solve the given question step by step. Make sure you think through every sentence you write,\
and that every sentence you write is coherent with all the previous sentences and the setting of the problem."
question = "Remove one ball at a time from a bag containing three black balls, four red balls and five white balls, and arrange all 12 balls in a horizontal line in the order in which they were removed. \
Note that the probability of an individual ball being taken out of the bag shall be equal. Find the probability p that none of the red balls are adjacent to each other."
solver_func(system_message, question)

<IPython.core.display.Latex object>

解答が明らかに長くなっていて、一番最後で約分ミスしている以外は合っている。改良版モデルでは、論理的整合性の面では改善されているが、計算の精度ではまだ問題があるようだ。そこで、エージェントを利用して計算の精度を高めてみる。

In [70]:
!pip install -q trulens 

In [72]:
!pip install -q trulens-apps-langchain

In [73]:
from langchain.chains import LLMMathChain
from langchain.agents import AgentType
from langchain.agents import Tool
from langchain.agents import initialize_agent
from trulens.apps.langchain import TruChain

llm_math_chain = LLMMathChain.from_llm(llm=model)
tools = [
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for calculation")
]

agent = initialize_agent(
    tools, model, agent=AgentType.OPENAI_FUNCTIONS, verbose=False
)



/Users/morikawakaion/anaconda3/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


🦑 TruSession initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `TruSession` to prevent this.


In [84]:
from langchain_core.prompts import ChatPromptTemplate

def solver_func_with_calculator(question):  
    system_template =  "You are a renowned mathematician. Solve the given question step by step. \
    Make sure you think through every sentence you write, and that every sentence you write is coherent with all the previous sentences\
    and the setting of the problem."
 #   question:```{question}```"
    prompt_template = ChatPromptTemplate.from_messages(
        [("system", system_template)]
    )
    chain = prompt_template | agent
    
    
    return display(Latex(agent.invoke([HumanMessage(content=question)])["output"]))

In [85]:


question = "Remove one ball at a time from a bag containing three black balls, four red balls and five white balls, and arrange all 12 balls in a horizontal line in the order in which they were removed. \
Note that the probability of an individual ball being taken out of the bag shall be equal. Find the probability p that none of the red balls are adjacent to each other."
solver_func_with_calculator(question)

<IPython.core.display.Latex object>

最後から2番目のイコールの部分で粗い丸め方をしているため、本当の答え（0.2545）からややずれているが、ひとつ前のコードのような致命的な計算ミスはしていないので、改善しているとは言える。